In [1]:
import glob
import os
import data_import
import jsonlines
from importlib import reload

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# Read and load PDF data

In [2]:
data_folder='../data/AMS/'
docs = glob.glob(data_folder+'*.pdf')   # Only get the PDFs in the directory

In [ ]:
pages=data_import.load_docs(docs=docs,
                            chunk_size=5000,
                            chunk_overlap=0,
                            file=data_folder+'ams_data_answers.jsonl')

# Initialize the llm

In [ ]:
import data_import
reload(data_import)

pages=data_import.read_docs(data_folder+'ams_data_answers.jsonl')

In [3]:
import openai
from langchain.llms import OpenAI

openai.api_key = os.getenv('OPENAI_API_KEY')

max_generated_tokens=516
temperature=0

# model_name='local-model'    # This can be anything. I recommend using the name of the loaded model from LM Studio.
# model_name='TheBloke/Mistral-7B-Instruct-v0.1-GGUF'
model='TheBloke/'
name='Llama-2-7B-Chat-GGUF'

base_url='http://localhost:1234/v1' # point to the local server

In [ ]:
llm = OpenAI(model_name=model+name,
             base_url=base_url,
             temperature=temperature,
             max_tokens=max_generated_tokens,
             tags=[model+name+'-'+str(temperature)+'-'+str(max_generated_tokens)]
            )

# Generate questions from data

In [ ]:
from importlib import reload
import gen_training_data
reload(gen_training_data)

training_data=gen_training_data.gen_q_from_context(pages[:100],
                                                   llm,
                                                   file=data_folder+'ams_data_train_'+name+'.jsonl')

# Format training_data for fine tuning

In [16]:
from importlib import reload
import gen_training_data
reload(gen_training_data)

format='LLM-generic'
tune_data=gen_training_data.format_dataset(format,
                                           file_in=data_folder+'ams_data_train_'+name+'.jsonl',
                                           file_out=data_folder+'ams_data_train_'+name+'-'+format+'-100.csv')

### Update to be in mistral format

In [ ]:
import pandas as pd
import glob
import os

# Get a list of all CSV files in the data subfolder
csv_files = glob.glob('data/**/*.csv', recursive=True)

for file in csv_files:
    if not file.endswith('-mistral.csv'):
        print(file)
        # Read the CSV file
        df = pd.read_csv(file)

        # Perform the string replacement
        df.iloc[:, 0] = df.iloc[:, 0].str.replace(r'### Human: (.*?)### Assistant:(.*)', r'<s>[INST] \1 [/INST] \2<s>', regex=True)

        # Write to file, append -mistral
        base = os.path.splitext(file)[0]
        df.to_csv(f'{base}-mistral.csv', index=False, mode='w')